1. Скачать видео:
https://www.youtube.com/watch?v=NdSqAAT28v0

In [2]:
!yt-dlp 'https://www.youtube.com/watch?v=NdSqAAT28v0'

[youtube] Extracting URL: https://www.youtube.com/watch?v=NdSqAAT28v0
[youtube] NdSqAAT28v0: Downloading webpage
[youtube] NdSqAAT28v0: Downloading ios player API JSON
[youtube] NdSqAAT28v0: Downloading player 4b63a6a1
[youtube] NdSqAAT28v0: Downloading m3u8 information
[info] NdSqAAT28v0: Downloading 1 format(s): 244+251
[download] Destination: Techno⧸Tech House Mix by DJ Haluk Arslan with Shadow Dancers [NdSqAAT28v0].f244.webm
[download] 100% of  275.72MiB in 00:00:34 at 7.90MiB/s0;33m00:000m
[download] Destination: Techno⧸Tech House Mix by DJ Haluk Arslan with Shadow Dancers [NdSqAAT28v0].f251.webm
[download] 100% of   74.04MiB in 00:00:07 at 9.59MiB/s0;33m00:000m
[Merger] Merging formats into "Techno⧸Tech House Mix by DJ Haluk Arslan with Shadow Dancers [NdSqAAT28v0].webm"
Deleting original file Techno⧸Tech House Mix by DJ Haluk Arslan with Shadow Dancers [NdSqAAT28v0].f251.webm (pass -k to keep)
Deleting original file Techno⧸Tech House Mix by DJ Haluk Arslan with Shadow Dancers [N

2. Средствами библиотек Python извлечь из видео кадры (в оригинале извлекался каждый третий кадр).
3. Средствами библиотек Python преобразовать разрешение кадров 1920*1080 -> 96*54 (можно другое соотношение).
4. Средствами библиотек Python перевести в черно-белое изображение (белый фон, черная фигура)
5. Обучить в Pytorch автоэнкодер (с помощью полносвязных слоев или, как в видео, сверточных) так, чтобы каждое изображение кодировалось вектором длины 128.
6. Перевести все изображения в последовательность 128-мерных векторов.
7. Обучить рекуррентную нейронную сеть (подобрать параметры самостоятельно) предсказывать по текущему следующий вектор.
8. С помощью обученной сети сгенерировать последовательность векторов.
9. Перевести с помощью декодера последовательность векторов в последовательность кадров.
10. Средствами библиотек Python преобразовать кадры в видео.